# Knowledge Based Filtering

지식 기반 필터링(Knowledge-based Filtering)"은 사용자가 제공한 명시적인 요구 사항을 바탕으로 아이템을 추천하는 시스템입니다. 이 방법은 사용자의 과거 행동이나 명확한 선호도가 부족할 때 유용하며, 사용자가 특정 요구 사항을 입력함으로써 추천을 받게 됩니다.

예를 들어, 영화 추천 시스템에서 사용자가 선호하는 장르, 감독, 배우, 또는 영화의 특정 기능(예: 상영 시간, 연도 등)에 대한 선호도를 입력하면, 시스템은 이 정보를 기반으로 영화를 필터링하고 추천합니다.

지식 기반 필터링의 예제 코드를 작성하기 전에, 우리는 몇 가지 가정을 해야 합니다. 예를 들어, 우리에게는 영화에 대한 정보가 담긴 movies 데이터프레임과 사용자가 요구하는 조건을 충족하는 영화를 추천하는 함수가 필요합니다.

In [1]:
import pandas as pd

# MovieLens 데이터셋 로드
movies = pd.read_csv('data/ml-latest-small/movies.csv')
ratings = pd.read_csv('data/ml-latest-small/ratings.csv')

In [2]:
movies.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [3]:
ratings.head(2)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247


In [4]:
# 장르를 기반으로 필터링
filterd_movies = movies['genres'].str.contains('Comedy')
movies[filterd_movies]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
6,7,Sabrina (1995),Comedy|Romance
...,...,...,...
9732,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi
9734,193571,Silver Spoon (2014),Comedy|Drama
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy


In [5]:
# 연도를 기반으로 필터링
filtered_movies = movies['title'].str.contains('\\(1995\\)')
movies[filtered_movies]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
8066,99130,Ice Cream Man (1995),Comedy|Horror
8836,132084,Let It Be Me (1995),(no genres listed)
8931,136297,Mortal Kombat: The Journey Begins (1995),Action|Animation
9453,167772,The Spirit of Christmas (1995),Animation|Comedy


In [6]:
# 평점 정보를 가진 영화 list
rated_movie_ids = ratings['movieId'].unique()
rated_movie_ids

array([     1,      3,      6, ..., 160836, 163937, 163981], dtype=int64)

In [7]:
movies[movies['movieId'].isin(rated_movie_ids)]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


### 장르와 연도를 기반으로 영화를 추천하는 함수

In [8]:
def recommend_movies(movies, ratings, preferred_genre=None, year=None):

    filtered_movies = movies.copy()

    # 장르를 기반으로 필터링. na=False : 결측치가 있는 데이터를 제외하고 필터링
    if preferred_genre:
        filtered_movies = filtered_movies[filtered_movies['genres'].str.contains(preferred_genre, na=False)]

    # 연도를 기반으로 필터링
    if year:
        year_pattern = f'\\({year}\\)'
        filtered_movies = filtered_movies[filtered_movies['title'].str.contains(year_pattern, regex=True, na=False)]
    
    # 평점 정보를 가진 영화만을 최종적으로 선택
    rated_movie_ids = ratings['movieId'].unique()
    filtered_movies = filtered_movies[filtered_movies['movieId'].isin(rated_movie_ids)]
    
    return filtered_movies

In [9]:
genre_list = sorted(list(set([genre for sublist in movies['genres'].str.split("|") for genre in sublist])))
print(genre_list)

['(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


In [10]:
# 사용자 입력 받기
genre = input("선호하는 장르를 입력하세요: \n{}".format(genre_list))
year = input("\n영화 연도를 입력하세요 (예: 1995): ")

선호하는 장르를 입력하세요: 
['(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'] Children

영화 연도를 입력하세요 (예: 1995):  1996


In [11]:
# year 입력이 숫자인지 확인하고, 숫자라면 int로 변환
if year.isdigit():
    year = int(year)
else:
    year = None

genre, year

('Children', 1996)

In [12]:
# 사용자가 선호하는 장르와 연도를 기반으로 영화 추천 받기
recommended_movies = recommend_movies(movies, ratings, preferred_genre=genre, year=year)

# 추천된 영화 출력
recommended_movies

,movieId,title,genres
78,87,Dunston Checks In (1996),Children|Comedy
95,107,Muppet Treasure Island (1996),Adventure|Children|Comedy|Musical
521,609,Homeward Bound II: Lost in San Francisco (1996),Adventure|Children
534,631,All Dogs Go to Heaven 2 (1996),Adventure|Animation|Children|Fantasy|Musical|R...
551,661,James and the Giant Peach (1996),Adventure|Animation|Children|Fantasy|Musical
559,673,Space Jam (1996),Adventure|Animation|Children|Comedy|Fantasy|Sc...
580,711,Flipper (1996),Adventure|Children
618,783,"Hunchback of Notre Dame, The (1996)",Animation|Children|Drama|Musical|Romance
629,801,Harriet the Spy (1996),Children|Comedy
635,808,Alaska (1996),Adventure|Children
